In [1]:
import pandas as pd

df = pd.read_csv("tubedata.csv", header=None)
df.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [2]:
from collections import defaultdict

station_dict = defaultdict(list)
station_dict_with_lines = defaultdict(dict)
zone_dict = defaultdict(set)

# get data row by row
for index, row in df.iterrows():
    start_station = row[0]
    end_station = row[1]
    act_cost = int(row[3])
    zone1 = row[4]
    zone2 = row[5]
    # station dict. of child station tuples (child name, cost from parent to the child)
    # f"Mile End": [("Stepney Green", 2), ("Wembley", 1)]g
    station_list = station_dict[start_station]
    station_list.append((end_station, act_cost))
    # the following two lines add the other direction of the tube ``step''
    station_list = station_dict[end_station]
    station_list.append((start_station, act_cost))
    
    line = row[2]
    station_list_with_lines = station_dict_with_lines[start_station]
    station_list_with_lines[end_station] = {"cost":act_cost, "line":line}
    station_list_with_lines = station_dict_with_lines[end_station]
    station_list_with_lines[start_station] = {"cost":act_cost,"line":line}
    # we add the main zone
    zone_dict[start_station].add(zone1)
    # we add the secondary zone
    if zone2 != "0":
        zone_dict[start_station].add(zone2)
        # if the secondary zone is not 0 it's the main zone for the ending station
        zone_dict[end_station].add(zone2)
    else:
        # otherwise the main zone for the ending station is the same as the starting station
        zone_dict[end_station].add(zone1)

**2.1. Implement DFS, BFS, and USC**

In [3]:
import queue

def PathExpander(nodeName, reached):
    path = (reached[nodeName]["actual cost"],[nodeName])
    parent = reached[nodeName]["parent"]
    while parent != None:
        path[1].append(parent)
        parent = reached[parent]["parent"]
    path[1].reverse()
    return path

def Expansion(graph, start, goal, node, Evaluation,val,reached):
    l = val
    for child in graph[node[2][0]]:
        l+=1
        results = Evaluation(graph, start, goal, node,child) ## Returns the priority and cost
        yield [l,(results[0],l,(child[0],results[1],graph[child[0]],results[2]))]

def BestFirstSearch(graph, start, goal, Evaluation):
    ##### (priorty, second priority, (name, cost, children, actual cost))
    node = (0,0,(start,0,dict(graph[start]),0))
    frontier = queue.PriorityQueue()
    frontier.put(node)
    reached = {start:{"cost":0,"parent":None,"actual cost":0}}
    val = 0
    while not(frontier.empty()):
        node = frontier.get()
        if node[2][0] == goal:
            return (*PathExpander(node[2][0],reached),val)
        for (val,child) in Expansion(graph, start, goal, node, Evaluation, val, reached):
            if not(child[2][0] in reached) or child[2][1]<reached[child[2][0]]["cost"]:
                reached[child[2][0]] = {"cost":child[2][1], "parent":node[2][0],"actual cost":child[2][3]}
                frontier.put(child)

In [4]:
def BreadthFirstEvaluation(graph, start, goal, node,child):
    new_cost = node[2][1] + 1
    actual_cost = node[2][3] + child[1]
    return (new_cost, new_cost,actual_cost) ## (priority,)

In [5]:
def DepthFirstEvaluation(graph, start, goal, node, child):
    new_cost = node[2][1] + 1
    actual_cost = node[2][3] + child[1]
    return (-new_cost, new_cost, actual_cost)

In [6]:
def UniformCostEvaluation(graph, start, goal, node, child):
    new_cost = node[2][3] + child[1]
    return (new_cost, new_cost,new_cost)

In [7]:
BestFirstSearch(station_dict, "Euston", "Victoria", BreadthFirstEvaluation)

(7,
 ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'],
 134)

In [8]:
BestFirstSearch(station_dict, "Euston", "Victoria", DepthFirstEvaluation)

(24,
 ['Euston',
  'Warren Street',
  'Goodge Street',
  'Tottenham Court Road',
  'Holborn',
  'Chancery Lane',
  "St. Paul's",
  'Bank/Monument',
  'Cannon Street',
  'Mansion House',
  'Blackfriars',
  'Temple',
  'Embankment',
  'Charing Cross',
  'Piccadilly Circus',
  'Green Park',
  'Victoria'],
 1222)

In [9]:
BestFirstSearch(station_dict, "Euston", "Victoria", UniformCostEvaluation)

(7,
 ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'],
 114)

In [10]:
def BreadthFirstSearch(graph, start, goal):
    return BestFirstSearch(graph,start,goal,BreadthFirstEvaluation)

def DepthFirstSearch(graph,start,goal):
    return BestFirstSearch(graph,start,goal,DepthFirstEvaluation)

def UniformCostSearch(graph,start,goal):
    return BestFirstSearch(graph,start,goal,UniformCostEvaluation)

In [11]:
print(BreadthFirstSearch(station_dict,"Euston", "Victoria"))
print(DepthFirstSearch(station_dict,"Euston", "Victoria"))
print(UniformCostSearch(station_dict,"Euston", "Victoria"))

(7, ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'], 134)
(24, ['Euston', 'Warren Street', 'Goodge Street', 'Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Cannon Street', 'Mansion House', 'Blackfriars', 'Temple', 'Embankment', 'Charing Cross', 'Piccadilly Circus', 'Green Park', 'Victoria'], 1222)
(7, ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'], 114)


**2.2 Compare DFS, BFS, and USC**

In [12]:
all_stations = list(station_dict)

In [13]:
bfs_results = {"Average Time Taken":0,"Average Nodes Expanded":0}
dfs_results = {"Average Time Taken":0,"Average Nodes Expanded":0}
ucs_results = {"Average Time Taken":0,"Average Nodes Expanded":0}
count=0
for i in all_stations:
    for j in all_stations:
        count+=1
        bfs = BreadthFirstSearch(station_dict, i,j)
        dfs = DepthFirstSearch(station_dict, i,j)
        ucs = UniformCostSearch(station_dict, i,j)
        bfs_results["Average Time Taken"] += bfs[0]
        bfs_results["Average Nodes Expanded"] += bfs[2]
        dfs_results["Average Time Taken"] += dfs[0]
        dfs_results["Average Nodes Expanded"] += dfs[2]
        ucs_results["Average Time Taken"] += ucs[0]
        ucs_results["Average Nodes Expanded"] += ucs[2]
bfs_results["Average Time Taken"] /= count
bfs_results["Average Nodes Expanded"] /= count
dfs_results["Average Time Taken"] /= count
dfs_results["Average Nodes Expanded"] /= count
ucs_results["Average Time Taken"] /= count
ucs_results["Average Nodes Expanded"] /= count
print(bfs_results)
print(dfs_results)
print(ucs_results)

{'Average Time Taken': 34.55187157037622, 'Average Nodes Expanded': 418.75708391770263}
{'Average Time Taken': 64.6174752522433, 'Average Nodes Expanded': 851.9059517163437}
{'Average Time Taken': 33.43824294331504, 'Average Nodes Expanded': 442.49498236679784}


In [14]:
print("BFS, DFS, and UCS on all problems on the state space\n")
print("BFS: ",bfs_results)
print("DFS: ",dfs_results)
print("UCS: ",ucs_results)

BFS, DFS, and UCS on all problems on the state space

BFS:  {'Average Time Taken': 34.55187157037622, 'Average Nodes Expanded': 418.75708391770263}
DFS:  {'Average Time Taken': 64.6174752522433, 'Average Nodes Expanded': 851.9059517163437}
UCS:  {'Average Time Taken': 33.43824294331504, 'Average Nodes Expanded': 442.49498236679784}


In [15]:
routes = (("Euston", "Victoria"),("Canada Water", "Stratford"), ("New Cross Gate", "Stepney Green"),
          ("Ealing Broadway", "South Kensington"), ("Baker Street", "Wembley Park"))
for start,goal in routes:
    bfs = BreadthFirstSearch(station_dict, start, goal)
    dfs = DepthFirstSearch(station_dict, start, goal)
    ucs = UniformCostSearch(station_dict, start, goal)
    print(f"{start} to {goal}\n")
    print("BFS:")
    print(f"Average Time Taken: {bfs[0]}")
    print(f"Route: {bfs[1]}")
    print(f"Nodes Expanded: {bfs[2]}\n")
    
    print("DFS:")
    print(f"Average Time Taken: {dfs[0]}")
    print(f"Route: {dfs[1]}")
    print(f"Nodes Expanded: {dfs[2]}\n")
    print("UCS:")
    print(f"Average Time Taken: {ucs[0]}")
    print(f"Route: {ucs[1]}")
    print(f"Nodes Expanded: {ucs[2]}\n\n\n")

Euston to Victoria

BFS:
Average Time Taken: 7
Route: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Nodes Expanded: 134

DFS:
Average Time Taken: 24
Route: ['Euston', 'Warren Street', 'Goodge Street', 'Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Cannon Street', 'Mansion House', 'Blackfriars', 'Temple', 'Embankment', 'Charing Cross', 'Piccadilly Circus', 'Green Park', 'Victoria']
Nodes Expanded: 1222

UCS:
Average Time Taken: 7
Route: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Nodes Expanded: 114



Canada Water to Stratford

BFS:
Average Time Taken: 15
Route: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Nodes Expanded: 143

DFS:
Average Time Taken: 20
Route: ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Nodes Expanded: 2630

UCS:
Average Time Taken

In [16]:
def ExpansionReversed(graph, start, goal, node, Evaluation,val,reached):
    l = val
    for child in graph[node[2][0]]:
        l-=1
        results = Evaluation(graph, start, goal, node,child) ## Returns the priority and cost
        yield [l,(results[0],l,(child[0],results[1],graph[child[0]],results[2]))]

def BestFirstSearchReversed(graph, start, goal, Evaluation):
    ##### (priorty, second priority, (name, cost, children, actual cost))
    node = (0,0,(start,0,dict(graph[start]),0))
    frontier = queue.PriorityQueue()
    frontier.put(node)
    reached = {start:{"cost":0,"parent":None,"actual cost":0}}
    val = 0
    while not(frontier.empty()):
        node = frontier.get()
        if node[2][0] == goal:
            return (*PathExpander(node[2][0],reached),-val)
        for (val,child) in ExpansionReversed(graph, start, goal, node, Evaluation, val, reached):
            if not(child[2][0] in reached) or child[2][1]<reached[child[2][0]]["cost"]:
                reached[child[2][0]] = {"cost":child[2][1], "parent":node[2][0],"actual cost":child[2][3]}
                frontier.put(child)

In [17]:
print("BFS Reversed: ",BestFirstSearchReversed(station_dict, "Ealing Broadway", "South Kensington",BreadthFirstEvaluation))
print("DFS Reversed: ",BestFirstSearchReversed(station_dict, "Ealing Broadway", "South Kensington",DepthFirstEvaluation))
print("UCS Reversed: ",BestFirstSearchReversed(station_dict, "Ealing Broadway", "South Kensington",UniformCostEvaluation))

BFS Reversed:  (20, ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'], 139)
DFS Reversed:  (57, ['Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Bond Street', 'Green Park', 'Victoria', 'Sloane Square', 'South Kensington'], 1259)
UCS Reversed:  (19, ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'], 122)


In [18]:
print("BFS: ",BestFirstSearch(station_dict, "Ealing Broadway", "South Kensington",BreadthFirstEvaluation))
print("DFS: ",BestFirstSearch(station_dict, "Ealing Broadway", "South Kensington",DepthFirstEvaluation))
print("UCS: ",BestFirstSearch(station_dict, "Ealing Broadway", "South Kensington",UniformCostEvaluation))

BFS:  (20, ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'], 135)
DFS:  (51, ['Ealing Broadway', 'West Acton', 'North Acton', 'East Acton', 'White City', "Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'Queensway', 'Lancaster Gate', 'Marble Arch', 'Bond Street', 'Oxford Circus', 'Piccadilly Circus', 'Charing Cross', 'Embankment', 'Waterloo', 'Kennington', 'Oval', 'Stockwell', 'Vauxhall', 'Pimlico', 'Victoria', 'Sloane Square', 'South Kensington'], 970)
UCS:  (19, ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'], 141)


**2.3 Extending the cost function**

In [19]:
import queue

def ExpansionWithLines(graph, start, goal, node, Evaluation,val,reached):
    l = val
    for child in graph[node[2][0]]:
        l+=1
        results = Evaluation(graph, start, goal, node,child) ## Returns the priority,cost and actual cost
        yield [l,(results[0],l,(child,results[1],graph[child],results[2],node[2][2][child]["line"]))]

def BestFirstSearchWithLines(graph, start, goal, Evaluation):
    ##### (priorty, second priority, (name, cost, children, actual cost, line))
    node = (0,0,(start,0,dict(graph[start]),0, None))
    frontier = queue.PriorityQueue()
    frontier.put(node)
    reached = {start:{"cost":0,"parent":None,"actual cost":0, "lines":[]}}
    val = 0
    while not(frontier.empty()):
        node = frontier.get()
        if node[2][0] == goal:
            return (*PathExpander(node[2][0],reached),val)
        for (val,child) in ExpansionWithLines(graph, start, goal, node, Evaluation, val, reached):
            if not(child[2][0] in reached) or child[2][1]<reached[child[2][0]]["cost"]:
                reached[child[2][0]] = {"cost":child[2][1], "parent":node[2][0],"actual cost":child[2][3], "lines":reached[node[2][0]]["lines"]+[child[2][4]]}
                frontier.put(child)

In [20]:
def UniformCostEvaluationWithLines(graph, start, goal, node, child):
    current_line = node[2][4]
    new_line = node[2][2][child]["line"]
    line_cost=0
    if current_line == None or new_line == current_line:
        line_cost=0
    else:
        line_cost=2
    new_cost = node[2][3] + node[2][2][child]["cost"] + line_cost
    return (new_cost, new_cost,new_cost)

In [21]:
def BreadthFirstEvaluationWithLines(graph, start, goal, node, child):
    current_line = node[2][4]
    new_line = node[2][2][child]["line"]
    line_cost=0
    if current_line == None or new_line == current_line:
        line_cost=0
    else:
        line_cost=2
    actual_cost = node[2][3] + node[2][2][child]["cost"] + line_cost
    new_cost = node[2][1] + 1
    return (new_cost, new_cost,actual_cost)

In [22]:
def DepthFirstEvaluationWithLines(graph, start, goal, node, child):
    current_line = node[2][4]
    new_line = node[2][2][child]["line"]
    line_cost=0
    if current_line == None or new_line == current_line:
        line_cost=0
    else:
        line_cost=2
    actual_cost = node[2][3] + node[2][2][child]["cost"] + line_cost
    new_cost = node[2][1] + 1
    return (-new_cost, new_cost,actual_cost)

In [23]:
BestFirstSearchWithLines(station_dict_with_lines, "Euston", "Victoria", UniformCostEvaluationWithLines)

(7, ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'], 62)

In [24]:
def UniformCostSearchWithLines(graph,start,goal):
    return BestFirstSearchWithLines(graph,start,goal,UniformCostEvaluationWithLines)

def BreadthFirstSearchWithLines(graph,start,goal):
    return BestFirstSearchWithLines(graph,start,goal,BreadthFirstEvaluationWithLines)

def DepthFirstSearchWithLines(graph,start,goal):
    return BestFirstSearchWithLines(graph, start, goal, DepthFirstEvaluationWithLines)

In [25]:
print(DepthFirstSearchWithLines(station_dict_with_lines, "Euston","Victoria"))

(39, ['Euston', 'Warren Street', 'Goodge Street', 'Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Cannon Street', 'Mansion House', 'Blackfriars', 'Temple', 'Embankment', 'Charing Cross', 'Piccadilly Circus', 'Green Park', 'Victoria'], 1028)


In [26]:
def returnLines(route):
    lines = []
    for i in range(len(route)-1):
        lines.append(station_dict_with_lines[route[i]][route[i+1]]["line"])
    return lines

In [27]:
ucs = UniformCostSearch(station_dict, "Marylebone", "Shepherd's Bush")
ucsl = UniformCostSearchWithLines(station_dict_with_lines, "Marylebone","Shepherd's Bush")
print("(Marylebone, Shepherd's Bush)\n")
print("UCS:")
print(ucs)
print("Lines: ",returnLines(ucs[1]))
print("\nUCS with Lines:")
print(ucsl)
print("Lines: ",returnLines(ucsl[1]))

(Marylebone, Shepherd's Bush)

UCS:
(13, ['Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush"], 277)
Lines:  ['Bakerloo', 'Hammersmith & City', 'District', 'District', 'Central', 'Central']

UCS with Lines:
(16, ['Marylebone', 'Edgware Road', 'Paddington', 'Royal Oak', 'Westbourne Park', 'Ladbroke Grove', 'Latimer Road', "Shepherd's Bush"], 193)
Lines:  ['Bakerloo', 'Hammersmith & City', 'Hammersmith & City', 'Hammersmith & City', 'Hammersmith & City', 'Hammersmith & City', 'Hammersmith & City']


In [28]:
dfs = DepthFirstSearch(station_dict, "Marylebone", "Shepherd's Bush")
dfsl = DepthFirstSearchWithLines(station_dict_with_lines, "Marylebone","Shepherd's Bush")
print("(Marylebone, Shepherd's Bush)\n")
print("DFS:")
print(dfs)
print("Lines: ",returnLines(dfs[1]))
print("\nDFS with Lines:")
print(dfsl)
print("Lines: ",returnLines(dfsl[1]))

(Marylebone, Shepherd's Bush)

DFS:
(13, ['Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush"], 54)
Lines:  ['Bakerloo', 'Hammersmith & City', 'District', 'District', 'Central', 'Central']

DFS with Lines:
(21, ['Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush"], 42)
Lines:  ['Bakerloo', 'Hammersmith & City', 'District', 'District', 'Central', 'Central']


In [29]:
bfs = BreadthFirstSearch(station_dict, "Marylebone", "Shepherd's Bush")
bfsl = BreadthFirstSearchWithLines(station_dict_with_lines, "Marylebone","Shepherd's Bush")
print("(Marylebone, Shepherd's Bush)\n")
print("BFS:")
print(bfs)
print("Lines: ",returnLines(bfs[1]))
print("\nBFS with Lines:")
print(bfsl)
print("Lines: ",returnLines(bfsl[1]))

(Marylebone, Shepherd's Bush)

BFS:
(13, ['Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush"], 221)
Lines:  ['Bakerloo', 'Hammersmith & City', 'District', 'District', 'Central', 'Central']

BFS with Lines:
(21, ['Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush"], 171)
Lines:  ['Bakerloo', 'Hammersmith & City', 'District', 'District', 'Central', 'Central']


**2.4 Heuristic Search**

In [30]:
int_list = ["1","2","3","4","5","6","7","8","9",]

def ZoneHeuristic(node, goal):

    node_lst = [int(i) if i in int_list else 0 for i in zone_dict[node]]
    goal_lst = [int(i) if i in int_list else 0 for i in zone_dict[goal]]
    node_zone = sum(node_lst)/len(node_lst)
    goal_zone = sum(goal_lst)/len(goal_lst)
    if node_zone == 0 or goal_zone == 0:
        return 0
    return abs(node_zone - goal_zone)

ZoneHeuristic("Chesham", "Euston")

0

In [31]:
def BestFirstSearchZoneHeuristic(graph, start, goal, Evaluation):
    ##### (priorty, second priority, (name, cost, children, actual cost))
    node = (0,0,(start,0,dict(graph[start]),0))
    frontier = queue.PriorityQueue()
    frontier.put(node)
    reached = {start:{"cost":0,"parent":None,"actual cost":0}}
    val = 0
    while not(frontier.empty()):
        node = frontier.get()
        if node[2][0] == goal:
            return (*PathExpander(node[2][0],reached),val)
        for (val,child) in ExpansionZoneHeuristic(graph, start, goal, node, Evaluation, val, reached):
            if not(child[2][0] in reached) or child[2][1]<reached[child[2][0]]["cost"]:
                reached[child[2][0]] = {"cost":child[2][1], "parent":node[2][0],"actual cost":child[2][3]}
                frontier.put(child)

In [32]:
def ExpansionZoneHeuristic(graph, start, goal, node, Evaluation,val,reached):
    l = val
    for child in graph[node[2][0]]:
        l+=1
        hv = ZoneHeuristic(child[0],goal)
        # print(child[0],zone_dict[child[0]],hv)
        results = Evaluation(graph, start, goal, node,child) ## Returns the priority and cost
        yield [l,(results[0]+hv,l,(child[0],results[1]+hv,graph[child[0]],results[2]))]


In [33]:
def HeuristicEvaluation(graph, start, goal, node,child):
    actual_cost = node[2][3] + child[1]
    return (0, 0,actual_cost) 

In [34]:
def ZoneHeuristicSearch(graph, start, end):
    return BestFirstSearchZoneHeuristic(graph, start, end,HeuristicEvaluation)

In [35]:
# print(UniformCostSearch(station_dict,"Euston", "Victoria"))
# print(ZoneHeuristicSearch(station_dict, "Euston", "Victoria"))
usch = UniformCostSearch(station_dict,"Shepherd's Bush", "Mile End")
zhs = ZoneHeuristicSearch(station_dict, "Shepherd's Bush", "Mile End")
print("(Shepherd's Bush, Mile End)\n")
print("Uniform Cost Search:")
print("Cost: ",usch[0])
print("Path: ",usch[1])
print("Nodes Expanded: ",usch[2])
print("\nZone Heuristic Search:")
print("Cost: ",zhs[0])
print("Path: ",zhs[1])
print("Nodes Expanded: ",zhs[2])

(Shepherd's Bush, Mile End)

Uniform Cost Search:
Cost:  28
Path:  ["Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'Queensway', 'Lancaster Gate', 'Marble Arch', 'Bond Street', 'Oxford Circus', 'Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Liverpool Street', 'Bethnal Green', 'Mile End']
Nodes Expanded:  563

Zone Heuristic Search:
Cost:  47
Path:  ["Shepherd's Bush", 'Goldhawk Road', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington', 'Sloane Square', 'Victoria', 'Pimlico', 'Vauxhall', 'Stockwell', 'Oval', 'Kennington', 'Elephant & Castle', 'Borough', 'London Bridge', 'Bermondsey', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End']
Nodes Expanded:  455
